In [8]:
-- Prolog term
data Term = V String        -- logic variable.       String 대문자로 시작
          | T String [Term] -- atom when empty list. String 소문자로 시작
      deriving (Eq,Ord,Show)

In [9]:
import Data.List
-- Prolog term처럼 보이게 좀더 예쁘게 출력할 수 있는 문자열로
pp (V x) = x
pp (T a []) = a
pp (T f ts) = f ++ (paren . concat . intersperse "," $ map pp ts )

paren s = "("++s++")"

In [10]:
-- 변수 X
V "X" 
pp(V "X")

V "X"

"X"

In [11]:
-- 아톰 int 
T "int" []
pp(T "int" [])

T "int" []

"int"

In [12]:
-- 복합항 f(int,int)
T "f" [T "int" [], T "int" []]
pp(T "f" [T "int" [], T "int" []])

T "f" [T "int" [],T "int" []]

"f(int,int)"

일치화 규칙

$\displaystyle
[a=a,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[f(t_1,\ldots,t_n)=f(t_1',\ldots,t_n'),~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_1',\cdots,t_n=t_n',u_2=u_2',~\cdots~],\sigma \\
[X=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[X=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{X\mapsto t\}\,[u_2=u_2',~\cdots~],\{X\mapsto t\}\uplus\sigma \qquad (X~\text{does not occur in}~ t) \\
[t=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[X=t,u_2=u_2',~\cdots~],\sigma
$

$\{X\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = \big[\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1,~\cdots,~\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1\big]$

$\{X\mapsto t\}\uplus\{X_1\mapsto t_1,~\cdots~,X_n\mapsto t_n \} = \big\{X\mapsto t, x_1\mapsto \{X\mapsto t\}t_1,~\cdots~,X_n\mapsto \{X\mapsto t\}t_n \big\}
$

In [13]:
occurs :: String -> Term -> Bool
occurs x (V x') = x==x'
occurs x (T _ ts) = any (occurs x) ts
-- occurs x (T _ ts) = or [occurs x t | t<-ts]

In [14]:
occurs "X" (T "f" [T "int" [], V "Y"])
occurs "X" (T "f" [T "int" [], V "X"])

False

True

In [15]:
subst :: (String,Term) -> Term -> Term
subst (x,t) u@(V y)  | x == y    = t
                     | otherwise = u
subst (x,t) (T f us) = T f [subst (x,t) u | u<-us]

In [16]:
-- {X |-> f(int,int)} f(X,X) ~~> f(f(int,int),f(int,int))
int = T "int" []

tm1 = T"f"[V"X",V"X"]
tm2 = subst ("X",T"f"[int,int]) (T"f"[V"X",V"X"])

pp tm1
pp tm2

"f(X,X)"

"f(f(int,int),f(int,int))"

일치화 규칙

$\displaystyle
[a=a,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[f(t_1,\ldots,t_n)=f(t_1',\ldots,t_n'),~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[t_1=t_1',\cdots,t_n=t_n',u_2=u_2',~\cdots~],\sigma \\
[X=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[u_2=u_2',~\cdots~],\sigma \\
[X=t,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
\{X\mapsto t\}\,[u_2=u_2',~\cdots~],\{X\mapsto t\}\uplus\sigma \qquad (X~\text{does not occur in}~ t) \\
[t=X,~u_2=u_2',~\cdots~],\sigma \rightsquigarrow
[X=t,u_2=u_2',~\cdots~],\sigma
$

$\{X\mapsto t\}\,[u_1=s_1,\cdots,u_n=s_n] = \big[\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1,~\cdots,~\{X\mapsto t\}\,u_1=\{X\mapsto t\}\,s_1\big]$

$\{X\mapsto t\}\uplus\{X_1\mapsto t_1,~\cdots~,X_n\mapsto t_n \} = \big\{X\mapsto t, x_1\mapsto \{X\mapsto t\}t_1,~\cdots~,X_n\mapsto \{X\mapsto t\}t_n \big\}
$

In [17]:
zip [1,2,3,4] [11,12,13,14]

[(1,11),(2,12),(3,13),(4,14)]

In [18]:
type Eqn = (Term,Term)       -- t1 = t2
type Subst = [(String,Term)] -- { X1 |-> t1, ..., Xn |-> tn }

-- 일치화 알고리듬을 위 규칙에 따라 작성
unif :: ([Eqn],Subst) -> ([Eqn],Subst)
unif ((T f ts, T g us): es, s)
       | f==g && length ts==length us  = unif (zip ts us ++ es, s)
unif ((V x   , V x'  ): es, s) | x==x' = unif (es, s)
unif ((V x   , t     ): es, s) | not(x `occurs` t) = unif (es', (x,t):s')
  where es' = [(subst (x,t) t1, subst (x,t) t2) | (t1,t2)<-es]
        s' = [(x',subst (x,t) t') | (x',t')<-s]
unif ((t     , V x   ): es, s) = unif ((V x,t):es, s)
unif (es, s) = (es, s)

In [19]:
t1 = T"c"[ V"X", T"c"[T"3"[],V"X"] ] -- c(X,c(3,X))
t2 = T"c"[ T"2"[], V"Y" ]            -- c(2,Y)

unif ([(t1,t2)], [])

([],[("Y",T "c" [T "3" [],T "2" []]),("X",T "2" [])])

In [20]:
t1 = T"c"[ V"X", T"c"[T"3"[],V"X"] ] -- c(X,c(3,X))
t2 = T"c"[ T"2"[], T"c"[V"X",V"Y"] ] -- c(2,c(X,X))

unif ([(t1,t2)], [])

([(T "3" [],T "2" []),(T "2" [],V "Y")],[("X",T "2" [])])

## 규칙 실행
규칙 실행은 여러 세트의 연립방정식을 만들어냄

규칙 실행의 결과 타입은 `[ [Eqn] ]`

In [21]:
ess1 = [ [1,1],  [2,2],  [3,3],  [4,4] ]
ess2 = [ [11,11],[22,22],[33,33],[44,44] ]

ess1
ess2

[es1++es2  | es1<-ess1, es2<-ess2]
length [es1++es2  | es1<-ess1, es2<-ess2]

[[1,1],[2,2],[3,3],[4,4]]

[[11,11],[22,22],[33,33],[44,44]]

[[1,1,11,11],[1,1,22,22],[1,1,33,33],[1,1,44,44],[2,2,11,11],[2,2,22,22],[2,2,33,33],[2,2,44,44],[3,3,11,11],[3,3,22,22],[3,3,33,33],[3,3,44,44],[4,4,11,11],[4,4,22,22],[4,4,33,33],[4,4,44,44]]

16

In [23]:
-- Prolog에서 =
(.=) :: Term -> Term -> [ [Eqn] ]
t1 .= t2  =  [ [(t1,t2)] ]

-- Prolog에서 ,
(.&) :: [ [Eqn] ] -> [ [Eqn] ] -> [ [Eqn] ]
ess1 .& ess2  = [es1++es2  | es1<-ess1, es2<-ess2]

-- Prolog에서 ;
(.|) :: [ [Eqn] ] -> [ [Eqn] ] -> [ [Eqn] ]
ess1 .| ess2  = ess1 ++ ess2

In [30]:
{-
 (X=john, Y=mary) ;
 (X=bob, Y=mary)  ;
 (X=jane, Y=john) ;
 (X=mary, Y=john) .
-}
x = V"X"
y = V"Y"
john = T "john" []
bob = T "bob" []
mary = T "mary" []
jane = T "jane" []

ess1 = ( (x.=john).&(y.=mary) ) .|
       ( (x.=bob).&(y.=mary) ) .|
       ( (x.=jane).&(y.=john) ) .|
       ( (x.=mary).&(y.=john) )

ess2 = ( (y.=john).&(x.=mary) ) .|
       ( (y.=bob).&(x.=mary) ) .|
       ( (y.=jane).&(x.=john) ) .|
       ( (y.=mary).&(x.=john) )

In [32]:
ess1
mapM_ print ess1

[[(V "X",T "john" []),(V "Y",T "mary" [])],[(V "X",T "bob" []),(V "Y",T "mary" [])],[(V "X",T "jane" []),(V "Y",T "john" [])],[(V "X",T "mary" []),(V "Y",T "john" [])]]

[(V "X",T "john" []),(V "Y",T "mary" [])]
[(V "X",T "bob" []),(V "Y",T "mary" [])]
[(V "X",T "jane" []),(V "Y",T "john" [])]
[(V "X",T "mary" []),(V "Y",T "john" [])]

In [33]:
ess1 .& ess2
mapM_ print (ess1 .& ess2)

[[(V "X",T "john" []),(V "Y",T "mary" []),(V "Y",T "john" []),(V "X",T "mary" [])],[(V "X",T "john" []),(V "Y",T "mary" []),(V "Y",T "bob" []),(V "X",T "mary" [])],[(V "X",T "john" []),(V "Y",T "mary" []),(V "Y",T "jane" []),(V "X",T "john" [])],[(V "X",T "john" []),(V "Y",T "mary" []),(V "Y",T "mary" []),(V "X",T "john" [])],[(V "X",T "bob" []),(V "Y",T "mary" []),(V "Y",T "john" []),(V "X",T "mary" [])],[(V "X",T "bob" []),(V "Y",T "mary" []),(V "Y",T "bob" []),(V "X",T "mary" [])],[(V "X",T "bob" []),(V "Y",T "mary" []),(V "Y",T "jane" []),(V "X",T "john" [])],[(V "X",T "bob" []),(V "Y",T "mary" []),(V "Y",T "mary" []),(V "X",T "john" [])],[(V "X",T "jane" []),(V "Y",T "john" []),(V "Y",T "john" []),(V "X",T "mary" [])],[(V "X",T "jane" []),(V "Y",T "john" []),(V "Y",T "bob" []),(V "X",T "mary" [])],[(V "X",T "jane" []),(V "Y",T "john" []),(V "Y",T "jane" []),(V "X",T "john" [])],[(V "X",T "jane" []),(V "Y",T "john" []),(V "Y",T "mary" []),(V "X",T "john" [])],[(V "X",T "mary" []),(

[(V "X",T "john" []),(V "Y",T "mary" []),(V "Y",T "john" []),(V "X",T "mary" [])]
[(V "X",T "john" []),(V "Y",T "mary" []),(V "Y",T "bob" []),(V "X",T "mary" [])]
[(V "X",T "john" []),(V "Y",T "mary" []),(V "Y",T "jane" []),(V "X",T "john" [])]
[(V "X",T "john" []),(V "Y",T "mary" []),(V "Y",T "mary" []),(V "X",T "john" [])]
[(V "X",T "bob" []),(V "Y",T "mary" []),(V "Y",T "john" []),(V "X",T "mary" [])]
[(V "X",T "bob" []),(V "Y",T "mary" []),(V "Y",T "bob" []),(V "X",T "mary" [])]
[(V "X",T "bob" []),(V "Y",T "mary" []),(V "Y",T "jane" []),(V "X",T "john" [])]
[(V "X",T "bob" []),(V "Y",T "mary" []),(V "Y",T "mary" []),(V "X",T "john" [])]
[(V "X",T "jane" []),(V "Y",T "john" []),(V "Y",T "john" []),(V "X",T "mary" [])]
[(V "X",T "jane" []),(V "Y",T "john" []),(V "Y",T "bob" []),(V "X",T "mary" [])]
[(V "X",T "jane" []),(V "Y",T "john" []),(V "Y",T "jane" []),(V "X",T "john" [])]
[(V "X",T "jane" []),(V "Y",T "john" []),(V "Y",T "mary" []),(V "X",T "john" [])]
[(V "X",T "mary" []),(V

필요에 따라서 새로운 논리변수를 만들어낼 수 있어야

In [36]:
[ unif (es,[]) | es <- ess1 .& ess2 ]
mapM_ print [ unif (es,[]) | es <- ess1 .& ess2]

[([(T "mary" [],T "john" []),(T "john" [],T "mary" [])],[("Y",T "mary" []),("X",T "john" [])]),([(T "mary" [],T "bob" []),(T "john" [],T "mary" [])],[("Y",T "mary" []),("X",T "john" [])]),([(T "mary" [],T "jane" []),(T "john" [],T "john" [])],[("Y",T "mary" []),("X",T "john" [])]),([],[("Y",T "mary" []),("X",T "john" [])]),([(T "mary" [],T "john" []),(T "bob" [],T "mary" [])],[("Y",T "mary" []),("X",T "bob" [])]),([(T "mary" [],T "bob" []),(T "bob" [],T "mary" [])],[("Y",T "mary" []),("X",T "bob" [])]),([(T "mary" [],T "jane" []),(T "bob" [],T "john" [])],[("Y",T "mary" []),("X",T "bob" [])]),([(T "bob" [],T "john" [])],[("Y",T "mary" []),("X",T "bob" [])]),([(T "jane" [],T "mary" [])],[("Y",T "john" []),("X",T "jane" [])]),([(T "john" [],T "bob" []),(T "jane" [],T "mary" [])],[("Y",T "john" []),("X",T "jane" [])]),([(T "john" [],T "jane" []),(T "jane" [],T "john" [])],[("Y",T "john" []),("X",T "jane" [])]),([(T "john" [],T "mary" []),(T "jane" [],T "john" [])],[("Y",T "john" []),("X",

([(T "mary" [],T "john" []),(T "john" [],T "mary" [])],[("Y",T "mary" []),("X",T "john" [])])
([(T "mary" [],T "bob" []),(T "john" [],T "mary" [])],[("Y",T "mary" []),("X",T "john" [])])
([(T "mary" [],T "jane" []),(T "john" [],T "john" [])],[("Y",T "mary" []),("X",T "john" [])])
([],[("Y",T "mary" []),("X",T "john" [])])
([(T "mary" [],T "john" []),(T "bob" [],T "mary" [])],[("Y",T "mary" []),("X",T "bob" [])])
([(T "mary" [],T "bob" []),(T "bob" [],T "mary" [])],[("Y",T "mary" []),("X",T "bob" [])])
([(T "mary" [],T "jane" []),(T "bob" [],T "john" [])],[("Y",T "mary" []),("X",T "bob" [])])
([(T "bob" [],T "john" [])],[("Y",T "mary" []),("X",T "bob" [])])
([(T "jane" [],T "mary" [])],[("Y",T "john" []),("X",T "jane" [])])
([(T "john" [],T "bob" []),(T "jane" [],T "mary" [])],[("Y",T "john" []),("X",T "jane" [])])
([(T "john" [],T "jane" []),(T "jane" [],T "john" [])],[("Y",T "john" []),("X",T "jane" [])])
([(T "john" [],T "mary" []),(T "jane" [],T "john" [])],[("Y",T "john" []),("X",T

In [37]:
result = [ unif (es,[]) | es <- ess1 .& ess2 ]
mapM_ print [sigma | ([],sigma)<-result]

[("Y",T "mary" []),("X",T "john" [])]
[("Y",T "john" []),("X",T "mary" [])]